In [19]:
from utils import *

## Tag Structure Parameters

In [20]:
plot = False
## Delay Line Parameters:
delta_L1 = (72 - 3) * 0.0254
delta_L2 = (72 - 48) * 0.0254
delta_L3 = (48 - 3) * 0.0254
expected_ratios = [
    delta_L1 / delta_L3,  # Ratio 1 (L1 to L3)
    delta_L3 / delta_L2,   # Ratio 2 (L3 to L2)
    delta_L1 / delta_L2,   # Ratio 2 (L3 to L2)
]

## Example input file
- -10 degree, slope: Data/pipeline/-10degree/9000-10000-150-100-m10-3-3.csv
- No radar file: Data/pipeline/noradar/0-0-0-0-0-0-1.csv

# 1. Radar Detection

In [21]:
threshold = 0.5
tolerance = 1

range_bins = np.array([3])

n10_directory_path = "./Data/pipeline/-10degree/"
n10_data_dict = read_data_dict(n10_directory_path)
n10_dist_data, n10_dist_snr = calculate_detection_rate_with_detectable_snr(n10_data_dict, tolerance, threshold, "distance", range_bins, expected_ratios, return_snr=True)

In [22]:
if n10_dist_data[0] == 1.0:
    print("Radar is detected!!!")

Radar is detected!!!


In [23]:
noradar_directory_path = "./Data/pipeline/noradar/"
noradar_data_dict = read_data_dict(noradar_directory_path)
noradar_dist_data, n10_dist_snr = calculate_detection_rate_with_detectable_snr(noradar_data_dict, tolerance, threshold, "distance", range_bins, expected_ratios, return_snr=True)
print("Radar Not Detected!!")

Error for 0-0-1-0-0-0.0-0: Score too low, signal not periodic!
Radar Not Detected!!


# 2. Radar Slope and Radar Angle of Arrival (AoA) Estimation
For detected radar signal, we need to further estimate it chirp slope and angle of arrival information

In [24]:
data_dict = read_data_dict(n10_directory_path)
for key in data_dict:
    data_obj = data_dict[key]
    theory_peak_1, theory_peak_2, theory_peak_3 = calculate_gt_frequency_peak(data_obj, delta_L1, delta_L2, delta_L3, d, c, speed_ratio)
    gt_peaks = theory_peak_1, theory_peak_2, theory_peak_3
    data = data_obj['data']
    gt_slope = data_obj['slope']
    gt_angle = data_obj['angle']
    timestamp = data[:, 0]
    amplitude = data[:, 1] * 1.0
    fs = len(timestamp) / ((timestamp[-1] - timestamp[0])) # timestamps in 1us
    _, _, valid_freq_list = find_beat_freq_triplet_with_expected_ratio(expected_ratios, tolerance, data_obj, radar_detection_mode = False, plot = False, return_snr = False, crop_to_list = True, fix_amplitude_order=True, noise_count = 10, noise_level = 7, window_length = 15)
    for predicted_peaks in valid_freq_list:
        if len(predicted_peaks) == 0:
            continue
        predicted_slope, predicted_angle = calculate_slope_and_angle(predicted_peaks, delta_L1, delta_L2, delta_L3, d, c, speed_ratio, frequency_offsets = [0, 1300, 0])
print(f"Predicted Slope: {predicted_slope}, Predicted Angle: {predicted_angle}")
print(f"GT Slope: {gt_slope}, GT Angle: {gt_angle}")

Predicted Slope: 6852467724883.765, Predicted Angle: -9.990164771550713
GT Slope: 6666666666666.667, GT Angle: -10
